# Classificação de Sentimentos com Redes Neurais Convolucionais

## Importações

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import spacy as sp
import string

import random
import re
import math
from bs4 import BeautifulSoup

In [12]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [15]:
from tensorflow.keras import layers
import tensorflow_datasets as tfds

## Preprocessamento dos dados

In [17]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']

In [20]:
test_data = pd.read_csv('../../_IAExpert_private/7. Processamento de Linguagem Natural com Deep LEarning/trainingandtestdata/test.csv', 
                        header=None, 
                        names=cols, 
                        engine="python", 
                        encoding='latin1')
print(test_data.shape)
test_data.sample(5, random_state=0)

(498, 6)


,sentiment,id,date,query,user,text
90,0,212,Mon May 25 17:23:55 UTC 2009,bailout,funky_old_man,Are YOU burning more cash $$$ than Chrysler an...
254,4,1019,Mon May 11 05:21:25 UTC 2009,lebron,undefeated310,Watching Lebron highlights. Damn that niggas good
283,2,2100,Sun May 17 17:30:03 UTC 2009,san francisco,bKnapp,"breakers. in San Francisco, CA http://loopt.us..."
443,4,2532,Sun Jun 07 01:13:23 UTC 2009,jquery,ghurson,"I'm ready to drop the pretenses, I am forever ..."
336,0,2185,Mon May 25 17:26:30 UTC 2009,insects,euthanasia86,Just got barraged by a horde of insects hungry...
